In [1]:
import cv2
import numpy as np
import pandas as pd
from Tracker import*
import datetime
from playsound import playsound
import threading

In [2]:
from ultralytics import YOLO
model = YOLO('yolov8s.pt') 

In [3]:
my_file = open("coco.txt", "r") 
data = my_file.read()
class_list = data.split("\n") 

In [4]:
def alert():
    threading.Thread(target = playsound, args=('.../text_to_speech.mp3',), daemon=True).start()

In [5]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)

In [ ]:
#area1 = [(596, 457),(79, 462),(71, 438),(580, 429)]
area1 = [(596, 457),(53, 465), (36, 406), (82, 406),(97, 440), (526, 432), (501, 398),(558, 400)]  
area2 = [(506, 424), (104, 430),(91, 402),(482, 396)] 
cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)
cap = cv2.VideoCapture(".../3.mp4")
count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 2 != 0:
        continue
    #frame = cv2.resize(frame,(1020,500))
    frame = frame[0:700, 400: 1000] 

    cv2.polylines(frame,[np.array(area1,np.int32)], True, (255,0,0), 2)
    cv2.polylines(frame,[np.array(area2,np.int32)], True, (255,0,0), 2)
    
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
#area2= [(596, 457),(79, 462),(71, 438),(580, 429)] 
area2 = [(596, 457),(53, 465), (36, 406), (82, 406),(97, 440), (526, 432), (501, 398),(558, 400)]  
area1 = [(506, 424), (104, 430),(91, 402),(482, 396)] 
cap = cv2.VideoCapture(".../3.mp4")
count = 0; j = 0
tracker = Tracker()

people_entering = {}
people_exiting = {}

entering = set(); nenter = 0
exiting = set(); nexit = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 4 != 0:
        continue
    #frame = cv2.resize(frame, (1020,500))
    frame = frame[0:700, 400: 1000] 
    results = model.predict(frame)
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype("float")
    list = []
    for index,row in px.iterrows():
        x1 = int(row[0]) #x coor of starting point 
        y1 = int(row[1]) #y coor of starting point
        x2 = int(row[2]) #x coor of ending point 
        y2 = int(row[3]) #y coor of ending point
        d = int(row[5])
        c = class_list[d] 
        if 'person' in c:
            list.append([x1, y1, x2, y2])
            
    bbox_id = tracker.update(list)
    for bbox in bbox_id:    
        x3, y3, x4, y4, id = bbox
        #Entry
        results = cv2.pointPolygonTest(np.array(area2,np.int32), ((x4, y4)), False)
        if results >= 0:
            people_entering[id] = (x4, y4)
            cv2.rectangle(frame,(x3,y3),(x4,y4), (0,0,255), 2) 
        if id in people_entering:
            results1 = cv2.pointPolygonTest(np.array(area1,np.int32), ((x4, y4)), False)
            if results1 >= 0:
                cv2.rectangle(frame,(x3,y3),(x4,y4), (0,255,0), 2) 
                cv2.circle(frame, (x4, y4), 4, (255,0,255), -1)
                cv2.putText(frame, str(id) ,(x3, y3),cv2.FONT_HERSHEY_COMPLEX,(0.7),(255,255,255),1)
                entering.add(id)
                if j % 100 == 0:
                    alert()
                j += 1
        #Exit
        results2 = cv2.pointPolygonTest(np.array(area1,np.int32), ((x4, y4)), False)
        if results2 >= 0:
            people_exiting[id] = (x4, y4)
            cv2.rectangle(frame,(x3,y3),(x4,y4), (0,255,0), 2) 
        if id in people_exiting:
            results3 = cv2.pointPolygonTest(np.array(area2,np.int32), ((x4, y4)), False)
            if results3 >= 0:
                cv2.rectangle(frame,(x3,y3),(x4,y4), (255,0,255), 2) 
                cv2.circle(frame, (x4, y4), 4, (255,0,255), -1)
                cv2.putText(frame, str(id) ,(x3, y3),cv2.FONT_HERSHEY_COMPLEX,(0.7),(255,255,255),1)
                exiting.add(id)
                
    nenter = len(entering); 
    nexit = len(exiting); 
    nppl = nenter - nexit
    if nenter - nexit == 0:
        cv2.circle(frame, (37, 54), 18, (0,255,0), -1)
    else:
        cv2.circle(frame, (37, 54), 18, (0,0,255), -1)

    cv2.polylines(frame,[np.array(area1,np.int32)], True, (255,0,0), 2)
    cv2.polylines(frame,[np.array(area2,np.int32)], True, (255,0,0), 2)

    current_datetime = datetime.datetime.now()
    time = current_datetime.strftime("%H:%M:%S")
    date = current_datetime.strftime("%Y-%m-%d")
    cv2.putText(frame, time ,(200, 678),cv2.FONT_HERSHEY_COMPLEX,(1),(0,0,0),2)
    cv2.putText(frame, date ,(360, 678),cv2.FONT_HERSHEY_COMPLEX,(1),(0,0,0),2)   

    cv2.putText(frame, "Entry: " + str(nenter) ,(54, 637),cv2.FONT_HERSHEY_COMPLEX,(1),(0,0,0),2)
    cv2.putText(frame, "Exit: " + str(nexit) ,(54, 678),cv2.FONT_HERSHEY_COMPLEX,(1),(0,0,0),2)        
    cv2.imshow("Frame", frame)
    if time == "00:00:00":
        file_path = ".../Tracks/" + date + '.txt'
        lines = ['Total People Entered today: ' + str(current_nenter), 'Total People Exited today: ' + str(current_nexit)]
        with open(file_path, 'w') as f:
            f.writelines('\n'.join(lines))
        current_nenter = nenter
        current_nexit = nexit
    if time == "00:00:02":
        nenter = 0
        nexit = 0
        exiting = set()
        entering = set()


    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)